# ELECTRONIC VOTING MACHINE 

In [ ]:
import cv2
import face_recognition
import numpy as np
import serial
import sqlite3
from sqlite3 import *
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk

x = -1
z = 1

def changetable(x, con):
    con.execute(f"UPDATE VOTERID SET CH = -1 WHERE ID = {x}")
    con.commit()

def checking(con, con1):
    global z

    def submit_data():
        global x
        voter_id = E1.get()

        # Check if the entered voter ID exists in the database
        if voter_id.isdigit(): 
            cur = con1.execute("SELECT ID FROM VOTERID")
            voter_ids = [row[0] for row in cur]
            if int(voter_id) in voter_ids:
                x = int(voter_id)
                app.destroy()
            else:
                messagebox.showerror("Voter ID Not Found", "The entered voter ID does not exist in the voter list.")
        else:
            messagebox.showerror("Invalid Input", "Please enter a valid voter ID .")

    def compare_faces(known_encodings, unknown_encoding):
        results = face_recognition.compare_faces(known_encodings, unknown_encoding)
        return any(results)

    def get_face_encodings_from_database(conn, voter_id):
        cursor = conn.execute("SELECT Encoding FROM VOTERID WHERE ID = ?", (voter_id,))
        encodings = [np.frombuffer(row[0], dtype=np.float64) for row in cursor]
        return encodings

    def get_voter_id_from_face(con1, face_encoding):
        cursor = con1.execute("SELECT ID, Encoding FROM VOTERID")
        for row in cursor:
            voter_id, known_encoding = row
            known_encoding = np.frombuffer(known_encoding, dtype=np.float64)
            if compare_faces([known_encoding], face_encoding):
                return voter_id
        return None

    def capture_face():
        video_capture = cv2.VideoCapture(0)
        ret, frame = video_capture.read()
        video_capture.release()
        cv2.imshow("Captured Face", frame)  # Display the captured face image to the user
        cv2.waitKey(0)  # Wait for key press to close the image window
        cv2.destroyAllWindows()  # Close the image window
        return frame

    def vote_registered():
        voter_details = get_voter_details(con1, x)
        if voter_details:
            voter_name = voter_details[0]
            voter_id = voter_details[1]
            messagebox.showinfo("Vote Registered", f"Your vote has been registered.\nName: {voter_name}\nID: {voter_id}")
        else:
            messagebox.showinfo("Vote Registered", "Your vote has been registered.")

    def face_verified(voter_name):
        messagebox.showinfo("Face Verification", f"FACE VERIFIED. \nVoter Name: {voter_name} \nPLEASE VOTE.")
        z = operate(con)
        if z != -1:
            vote_registered()
            changetable(x, con1)  # Change CH value to -1 after successful vote
        else:
            messagebox.showinfo("Vote Aborted", "You have chosen not to vote.")

    while z == 1:
        mylist = []
        app = Tk()
        app.title("VOTING_MACHINE")
        app.geometry('1600x1000')
        label_font = ('Arial', 25)

        labelText = StringVar()
        labelText.set("Cast Your Vote")
        label1 = Label(app, textvariable=labelText, height=4,font=label_font)
        label1.pack()

       

        L1 = Label(app, text="Enter Your I.D",font=label_font)
        L1.pack()
        E1 = Entry(app, bd=40,font=label_font)
        E1.pack()
        button1 = Button(app, text="SUBMIT", command=submit_data, width=20,font=label_font)
        button1.pack(padx=15, pady=15)
        
        labelText1 = StringVar()
        labelText1.set("\n1 - PARTY 1\n2 - PARTY 2\n3 - PARTY 3\n4 - NOTA  !\n ")
        label1 = Label(app, textvariable=labelText1, height=4,font=label_font)
        label1.pack()

        app.mainloop()

        cur = con1.execute("SELECT ID FROM VOTERID")
        cur2 = con1.execute("SELECT CH FROM VOTERID")

        flag = 0
        for i in cur:
            if i[0] == x:
                if [y[0] for y in cur2][x - 1] == 0:
                    flag = 1

        if flag == 1:
            messagebox.showinfo("Face Verification", "Please show your face for verification.")
            face_frame = capture_face()
            face_encodings = face_recognition.face_encodings(face_frame)

            if not face_encodings:  # Check if no faces were found
                messagebox.showerror("Face Verification", "No face detected. Try again.")
                z = 1
                continue

            face_encoding = face_encodings[0]

            verified_voter_id = get_voter_id_from_face(con1, face_encoding)
            if verified_voter_id and verified_voter_id == x:
                cursor = con1.execute("SELECT Name FROM VOTERID WHERE ID = ?", (x,))
                voter_name = cursor.fetchone()[0]
                face_verified(voter_name)
            else:
                messagebox.showerror("Face Verification", "Face not recognized for the provided voter ID. Try again.")
                z = 1
        else:
            messagebox.showerror("Verification Failed", "Your vote has already been recorded")
            z = 1

def operate(con):
    i = 0
    while i < 1:
        d = a.read().decode('utf-8')
        if d == "*":
            con.execute("UPDATE PP SET VOTES = VOTES + 1 WHERE ID = 1")
            con.commit()
            result = printdb(con)
            i += 1
            return i
        elif d == "#":
            con.execute("UPDATE PP SET VOTES = VOTES + 1 WHERE ID = 2")
            con.commit()
            result = printdb(con)
            i += 1
            return i
        elif d == "$":
            con.execute("UPDATE PP SET VOTES = VOTES + 1 WHERE ID = 3")
            con.commit()
            result = printdb(con)
            i += 1
            return i
        elif d == "&":
            con.execute("UPDATE PP SET VOTES = VOTES + 1 WHERE ID = 4")
            con.commit()
            result = printdb(con)
            i += 1
            return i

def get_voter_details(conn, voter_id):
    cursor = conn.execute("SELECT Name, ID FROM VOTERID WHERE ID = ?", (voter_id,))
    row = cursor.fetchone()
    if row:
        return row
    else:
        return None

def printdb(con):
    cursor = con.execute("SELECT id, name, VOTES FROM PP")
    print("ID\t", "NAME\t", "VOTES\n")
    for row in cursor:
        print(row[0], "\t", row[1], "\t", row[2], "\n")

a = serial.Serial('COM4', 9600)
con1 = connect('people.db')
con = connect('votes.db')
checking(con, con1)